In [ ]:
%autosave 0
import numpy as np 
import pandas as pd 
from pandas import read_csv, DataFrame
from math import sqrt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics.scorer import make_scorer

from sklearn.preprocessing import PolynomialFeatures 
from sklearn.pipeline import Pipeline

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Activation
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import seaborn as sns
import matplotlib.pyplot as plt

# ignore Warning
import warnings
warnings.filterwarnings("ignore") 



In [ ]:
#### Mettre la variable sur True si recherche des paramétres opti, mais l'opération dure environ 45 minutes
lookForOptimiz = True

In [ ]:
X_train = read_csv('X_train.csv', index_col=0)
X_val = read_csv('X_val.csv', index_col=0)
test_df = read_csv('X_test.csv', index_col=0)
y_train_log = read_csv('y_train.csv', index_col=0)
y_val_log = read_csv('y_val.csv', index_col=0)

# Output Scaler

In [ ]:
scaler_Y = StandardScaler()
scaler_Y.fit(y_train_log)

y_train_log_scaled = scaler_Y.transform(y_train_log)
y_val_log_scaled = scaler_Y.transform(y_val_log)

In [ ]:
X_train = X_train.values.astype(float)
X_val = X_val.values.astype(float)
test = test_df.values.astype(float)

In [ ]:
nb_features = 75 # 24
X_train = X_train[:,:nb_features]
X_val = X_val[:,:nb_features]
test = test[:,:nb_features]

In [ ]:
seed = 42
np.random.seed(seed)

# Recherche des paramétres opti du réseau

In [ ]:
sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)

In [ ]:
def create_model(neurons=100, l1=0.01):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], kernel_regularizer=regularizers.l1(l1)))
    model.add(Activation('relu')) # activity_regularizers = regularizers.l2(0.01)
    model.add(Dense(1, kernel_initializer='normal',kernel_regularizer=regularizers.l1(l1)))
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

Creation of a custom scoring method

In [ ]:
def RMSLE(y_true_log_scaled, y_pred_log_scaled):
    y_pred_log = scaler_Y.inverse_transform(y_pred_log_scaled)
    y_true_log = scaler_Y.inverse_transform(y_true_log_scaled)
    return np.sqrt(MSE(y_pred_log, y_true_log))    
score = make_scorer(RMSLE, greater_is_better=False)

In [ ]:
def BestParams():
    estimators = []
    #estimators.append(('pf', PolynomialFeatures(interaction_only=True,include_bias=False)))
    #estimators.append(('ss', StandardScaler()))
    estimators.append(('nn', KerasRegressor(build_fn=create_model, epochs=10, batch_size=5, neurons=200, verbose=0)))
    
    nn_pipe = Pipeline(estimators)
    print(nn_pipe.get_params().keys())
    
    nn_param_grid = {'nn__epochs':[1000],'nn__batch_size':[100],
                     'nn__neurons':[60,70],'nn__l1':[0.005,0.001,0.002]}
    
    gs_nn = GridSearchCV(nn_pipe, nn_param_grid,verbose=0, cv=3, scoring=score)
    print('Ok')
    gs_nn.fit(X_train, y_train_log_scaled)
    
    return gs_nn.best_params_

In [ ]:
X_train.shape, y_train.shape

In [ ]:
if lookForOptimiz : 
    best = BestParams()
    print(best)

# Creation du modèle

In [ ]:
model = create_model(neurons=60, l1=0.005)

In [ ]:
m_fti = model.fit(X_train, y_train_log_scaled, epochs=1000, batch_size=100)

In [ ]:
DataFrame(m_fti.history).plot()
plt.grid = True
plt.show()

In [ ]:
X_val.shape

# Validation score

In [ ]:
pred_log_scaled = model.predict(X_val)
pred_log = scaler_Y.inverse_transform(pred_log_scaled)

In [ ]:
RMSE = np.sqrt(MSE(pred_log,y_val_log))
print(RMSE.round(4))

# Train score

In [ ]:
pred_log_scaled = model.predict(X_train)
pred_log = scaler_Y.inverse_transform(pred_log_scaled)

In [ ]:
RMSE = np.sqrt(MSE(pred_log,y_train_log))
print(RMSE.round(4))

# Kaggle prediction

In [ ]:
pred_kagg_log_scaled = model.predict(test)

In [ ]:
pred_kagg_log_scaled.shape

In [ ]:
pred_kagg_log = scaler_Y.inverse_transform(pred_kagg_log_scaled)
pred_kagg = np.exp(pred_kagg_log).reshape(-1)

In [ ]:
pred_kagg


In [ ]:
submission = DataFrame({"SalePrice": pred_kagg}, index=test_df.index)
submission.to_csv('new_kagg10.csv')